In [1]:
import bs4
import requests
import time
import random as ran
import sys
import pandas as pd

In [2]:
url = 'https://www.imdb.com/search/title?release_date=2018&sort=boxoffice_gross_us,desc&start=1'

url_response = requests.get(url).text
parser_html = bs4.BeautifulSoup(url_response,'html.parser')

In [3]:
movie_blocks = parser_html.findAll('div',{'class':'lister-item-content'})

In [4]:
movie_name = movie_blocks[0].find('a').get_text() 

movie_rating = float(movie_blocks[0].find('div',{'class':'inline-block ratings-imdb-rating'}).get('data-value'))

movie_year = int(movie_blocks[0].find('span',{'class': 'lister-item-year'}).contents[0][1:-1]) 


In [5]:
def scrap_mblock(movie_block):
    
    movie_data ={}
  
    movie_data['name'] = movie_block.find('a').get_text() 

    movie_data['rating'] = float(movie_block.find('div',{'class':'inline-block ratings-imdb-rating'}).get('data-value')) 

    movie_data['year'] = str(movie_block.find('span',{'class': 'lister-item-year'}).contents[0][1:-1])
    
    return movie_data

In [6]:
def scrap_movie_page(movie_blocks):
    
    page_movie_data = []
    num_blocks = len(movie_blocks)
    
    for block in range(num_blocks):
        page_movie_data.append(scrap_mblock(movie_blocks[block]))
    
    return page_movie_data


In [7]:
def scrap_web(link,t_count):

    base_url = link
    target = t_count
    
    current_mcount_start = 0
    current_mcount_end = 0
    remaining_mcount = target - current_mcount_end 
    
    new_page_number = 1
    
    movie_data = []
    
    
    while remaining_mcount > 0:

        url = base_url + str(new_page_number)
        
        source = requests.get(url).text
        soup = bs4.BeautifulSoup(source,'html.parser')
        
        movie_blocks = soup.findAll('div',{'class':'lister-item-content'})
        
        movie_data.extend(scrap_movie_page(movie_blocks))   
        
        current_mcount_start = int(soup.find("div", {"class":"nav"}).find("div", {"class": "desc"}).contents[1].get_text().split("-")[0])

        remaining_mcount = target - current_mcount_end
        
        print('\r' + "currently scraping movies from page : " + str(current_mcount_start) , end ="")
        new_page_number = current_mcount_start + 1
        
        time.sleep(ran.randint(0, 10))
    
    return movie_data

In [ ]:
base_scraping_link = "https://www.imdb.com/search/title?release_date=2018-01-01,2018-12-31&sort=boxoffice_gross_us,desc&start="

top_movies = 1000 
films = []

films = scrap_web(base_scraping_link,int(top_movies))

print('\r'+"List of top " + str(top_movies) +" movies:" + "\n", end="\n")
pd.DataFrame(films)

currently scraping movies from page : 13

In [ ]:
dataframe_csv = pd.DataFrame(films)

In [ ]:
imdb_movies = pd.DataFrame(dataframe_csv)
filename = 'imdb_movies.csv'
imdb_movies.to_csv(filename)